In [ ]:
from czibench.datasets.sc import SingleCellDataset
from czibench.datasets.types import Organism
from czibench.runner import ContainerRunner
from czibench.tasks.sc import ClusteringTask, EmbeddingTask

dataset = SingleCellDataset(
    "example.h5ad",
    organism=Organism.HUMAN,
)

runner = ContainerRunner(
    image="czibench-scvi:latest",
    gpu=True,
)

dataset = runner.run(dataset)

task = ClusteringTask(label_key="cell_type")
dataset, clustering_results = task.run(dataset)

task = EmbeddingTask(label_key="cell_type")
dataset, embedding_results = task.run(dataset)

In [2]:
clustering_results

In [3]:
embedding_results

In [4]:
import scanpy as sc
sc.tl.umap(dataset.adata)
sc.pl.umap(dataset.adata, color="cell_type")